# Bayes Classifiers

How can P(X|C) be modelled when the input features are not independent of each other?

There are multiple ways this can be done, for example: 
- Can use full covariance matrix 
- Use a Hidden Markov Model
- Create a Bayes Net


### Multivariate Gaussian distributions are not independent

#add more info here


In [1]:
import numpy as np
from datetime import datetime
from scipy.stats import norm
from scipy.stats import multivariate_normal as mvn

import sys
sys.path.append('../')
from utils import get_mnist_data


# Implementation

In general, the implementation of a Bayes Classifier is much the same as the Naive Bayes however, involves using the covariance of each value of X. Each value of X has to be transposed to provide a DxD matrix otherwise it will produce an NxN matrix. Numpy uses n-1 in covariance to avoid bias.

In [13]:
class Bayes(object):
    def fit(self, X,Y, smoothing=10e-3):
        N,D = X.shape
        self.gaussians = dict()
        self.priors = dict()
        labels = set(Y)
        for c in labels:
            current_x = X[Y == c]
            self.gaussians[c] = {
                'mean': current_x.mean(axis=0),
                'cov': np.cov(current_x.T) + np.eye(D) + smoothing
            }
            self.priors[c] = float(len(Y[Y==c]))/len(Y)
    
    
    def predict(self,X):
        N,D = X.shape
        K = len(self.gaussians)
        Yhat = np.zeros((N,K))
        for c, n in self.gaussians.items():
            mean, cov = n['mean'],n['cov']
            Yhat[:,c] = mvn.logpdf(X, mean=mean, cov=cov) + np.log(self.priors[c])
            
        return np.argmax(Yhat,axis=1)
    
    def score(self,X,Y):
        Yhat = self.predict(X)
        return np.mean(Y==Yhat)
    

In [14]:
if __name__=='__main__':
    fn = '../mnist/train.csv'
    X,Y = get_mnist_data(fn,10000)
    Ntrain = int(len(Y)/2)
    Xtrain, Ytrain = X[:Ntrain], Y[:Ntrain]
    Xtest, Ytest = X[Ntrain:], Y[Ntrain:]
    
    model = Bayes()
    
    # time how long it takes to fit the model
    t0 = datetime.now()
    model.fit(Xtrain,Ytrain)
    print('Training time:',(datetime.now() - t0))
    
    
    # time how long it takes to take the training accuracy
    t0 = datetime.now()
    print('Training accuracy:', model.score(Xtrain,Ytrain))
    print('Time taken to compute train accuracy of train size:',len(Ytrain),'::', (datetime.now()-t0))
    
    #time how long it takes to get the test accuracy
    t0 = datetime.now()
    print('Testing accuracy:',model.score(Xtest,Ytest))
    print('Time taken to compute testing accuracy of sample size:',len(Ytest),'::',(datetime.now()-t0))

Training time: 0:00:00.314981
Training accuracy: 0.9066
Time taken to compute train accuracy of train size: 5000 :: 0:00:04.399151
Testing accuracy: 0.8728
Time taken to compute testing accuracy of sample size: 5000 :: 0:00:05.015478
